<a href="https://colab.research.google.com/github/fagonzalezo/qmc/blob/master/examples/TPU-QM-DM-mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Install qmc if running in Google Colab

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    !pip install git+https://github.com/fagonzalezo/qmc.git
else:
    import sys
    sys.path.insert(0, "../")

  Cloning https://github.com/fagonzalezo/qmc.git to /tmp/pip-req-build-982t6st3
  Running command git clone -q https://github.com/fagonzalezo/qmc.git /tmp/pip-req-build-982t6st3
  Created wheel for qmc: filename=qmc-0.0.1-cp37-none-any.whl size=11694 sha256=8a185179a4c05e7434b83e44d8e8cef0b4ccd24cd90871ba037e116c15a86da2
  Stored in directory: /tmp/pip-ephem-wheel-cache-pgtsgvms/wheels/ff/c2/53/58cb50bb85949fb1dcda723a7c1f48a590515dee0aa40a4cc2
Successfully built qmc


In [8]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import qmc.tf.layers as layers
import qmc.tf.models as models

In [9]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Running on TPU  ['10.4.189.194:8470']
INFO:tensorflow:Initializing the TPU system: grpc://10.4.189.194:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.4.189.194:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [20]:
BATCH_SIZE = 64 * tpu_strategy.num_replicas_in_sync
training_images_file   = 'gs://mnist-public/train-images-idx3-ubyte'
training_labels_file   = 'gs://mnist-public/train-labels-idx1-ubyte'
validation_images_file = 'gs://mnist-public/t10k-images-idx3-ubyte'
validation_labels_file = 'gs://mnist-public/t10k-labels-idx1-ubyte'
def read_label(tf_bytestring):
    label = tf.io.decode_raw(tf_bytestring, tf.uint8)
    label = tf.reshape(label, [])
    label = tf.one_hot(label, 10)
    return label
  
def read_image(tf_bytestring):
    image = tf.io.decode_raw(tf_bytestring, tf.uint8)
    image = tf.cast(image, tf.float32)/256.0
    image = tf.reshape(image, [28*28])
    return image
  
def load_dataset(image_file, label_file):
    imagedataset = tf.data.FixedLengthRecordDataset(image_file, 28*28, header_bytes=16)
    imagedataset = imagedataset.map(read_image, num_parallel_calls=16)
    labelsdataset = tf.data.FixedLengthRecordDataset(label_file, 1, header_bytes=8)
    labelsdataset = labelsdataset.map(read_label, num_parallel_calls=16)
    dataset = tf.data.Dataset.zip((imagedataset, labelsdataset))
    return dataset 
  
def get_training_dataset(image_file, label_file, batch_size):
    dataset = load_dataset(image_file, label_file)
    dataset = dataset.cache()  # this small dataset can be entirely cached in RAM
    dataset = dataset.shuffle(5000, reshuffle_each_iteration=True)
    dataset = dataset.repeat()
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(-1)  # fetch next batches while training on the current one (-1: autotune prefetch buffer size)
    return dataset
  
def get_validation_dataset(image_file, label_file):
    dataset = load_dataset(image_file, label_file)
    dataset = dataset.cache() # this small dataset can be entirely cached in RAM
    dataset = dataset.batch(10000)
    return dataset

# instantiate the datasets
training_dataset = get_training_dataset(training_images_file, training_labels_file, BATCH_SIZE)
validation_dataset = get_validation_dataset(validation_images_file, validation_labels_file)

In [21]:
with tpu_strategy.scope(): # creating the model in the TPUStrategy scope means we will train the model on the TPU
    model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(10),
    tf.keras.layers.Softmax()
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(0.001),
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.CategoricalAccuracy()],
    )
EPOCHS = 10
steps_per_epoch = 60000//BATCH_SIZE  # 60,000 items in this dataset
print("Steps per epoch: ", steps_per_epoch)
  
history = model.fit(training_dataset,
                    steps_per_epoch=steps_per_epoch, epochs=EPOCHS,
                    validation_data=validation_dataset, validation_steps=1)

Steps per epoch:  117
Epoch 1/10


115/117 [============================>.] - ETA: 0s - loss: 1.9078 - categorical_accuracy: 0.6519WARNING:tensorflow:Model was constructed with shape (None, 28, 28) for input KerasTensor(type_spec=TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name='flatten_4_input'), name='flatten_4_input', description="created by layer 'flatten_4_input'"), but it was called on an input with incompatible shape (None, 784).


117/117 [==============================] - 6s 30ms/step - loss: 1.9031 - categorical_accuracy: 0.6559 - val_loss: 1.5721 - val_categorical_accuracy: 0.9108
Epoch 2/10
117/117 [==============================] - 2s 19ms/step - loss: 1.5672 - categorical_accuracy: 0.9159 - val_loss: 1.5477 - val_categorical_accuracy: 0.9276
Epoch 3/10
117/117 [==============================] - 2s 19ms/step - loss: 1.5462 - categorical_accuracy: 0.9284 - val_loss: 1.5360 - val_categorical_accuracy: 0.9350
Epoch 4/10
117/117 [==============================] - 2s 19ms/step - loss: 1.5332 - categorical_accuracy: 0.9391 - val_loss: 1.5285 - val_categorical_accuracy: 0.9409
Epoch 5/10
117/117 [==============================] - 3s 22ms/step - loss: 1.5248 - categorical_accuracy: 0.9453 - val_loss: 1.5228 - val_categorical_accuracy: 0.9452
Epoch 6/10
117/117 [==============================] - 2s 19ms/step - loss: 1.5191 - categorical_accuracy: 0.9503 - val_loss: 1.5186 - val_categorical_accuracy: 0.9480
Epoch 7/1

In [28]:
input_dim = 784
component_dim = 128
gamma = 2**-5
num_eig = 80
random_state = 0
LEARNING_RATE = 0.001
LEARNING_RATE_EXP_DECAY = 0.6 if tpu_strategy.num_replicas_in_sync == 1 else 0.7
lr_decay = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: LEARNING_RATE * LEARNING_RATE_EXP_DECAY**epoch,
    verbose=True)

with tpu_strategy.scope(): # creating the model in the TPUStrategy scope means we will train the model on the TPU
    inputs = tf.keras.Input(shape=(784,))
    fm_x = layers.QFeatureMapRFF(784, dim=component_dim , gamma=gamma, random_state=random_state)
    psi_x = fm_x(inputs)
    ones = tf.ones_like(inputs[:, 0:1])
    rho_x = tf.keras.layers.concatenate((ones, psi_x), axis=1)
    rho_x = tf.expand_dims(rho_x, axis=-1)
    qmdmc = layers.QMeasureDMClassifEig(dim_x=component_dim , dim_y=10, eig_out=num_eig * 2, num_eig=num_eig)
    rho_y = qmdmc(rho_x)
    y_w = rho_y[:, 0, :] # shape (b, d_in)
    y_v = rho_y[:, 1:, :] # shape (b, dim_x, d_in)
    probs = tf.einsum('...j,...ij,...ij->...i', y_w, y_v, tf.math.conj(y_v))
    qmdmc = tf.keras.Model(inputs=inputs, outputs=probs)
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001, decay=1e-6)
    qmdmc.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer='adam', metrics=['accuracy'])

EPOCHS = 10
steps_per_epoch = 60000//BATCH_SIZE  # 60,000 items in this dataset
print("Steps per epoch: ", steps_per_epoch)
  
history = qmdmc.fit(training_dataset,
                    steps_per_epoch=steps_per_epoch, epochs=EPOCHS,
                    validation_data=validation_dataset, validation_steps=1,
                    callbacks=[lr_decay])

Steps per epoch:  117
Epoch 1/10

Epoch 00001: LearningRateScheduler reducing learning rate to 0.001.
117/117 [==============================] - 10s 41ms/step - loss: 2.0566 - accuracy: 0.3191 - val_loss: 1.4221 - val_accuracy: 0.6684
Epoch 2/10

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0007.
117/117 [==============================] - 3s 22ms/step - loss: 1.3502 - accuracy: 0.6869 - val_loss: 1.1410 - val_accuracy: 0.7287
Epoch 3/10

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00049.
117/117 [==============================] - 3s 22ms/step - loss: 1.1275 - accuracy: 0.7365 - val_loss: 1.0572 - val_accuracy: 0.7521
Epoch 4/10

Epoch 00004: LearningRateScheduler reducing learning rate to 0.00034299999999999993.
117/117 [==============================] - 3s 22ms/step - loss: 1.0554 - accuracy: 0.7599 - val_loss: 1.0180 - val_accuracy: 0.7620
Epoch 5/10

Epoch 00005: LearningRateScheduler reducing learning rate to 0.00024009999999999995.
117/117 [===